# Validation of Your Customized Model
Our evaluation framework is as follows. 

You only need to define your own Customized Model that extracts and outputs ECG features (with shape: batch × d_model or batch × seq_len × d_model). 
![framework](../image/framework.png)

### Step 1. Define Your Custom Model
Create a simple MLP model

In [ ]:
import math
import numpy as np
from torch import nn
import torch.nn.functional as F


class CustomizedModel(nn.Module):
    def __init__(self,args=None, input_channels=1, d_model=32, **kwargs):
        super(CustomizedModel, self).__init__()
        self.args = args
        self.input_channels = input_channels
        self.d_model = d_model

        self.encoder = nn.Sequential(nn.Linear(input_channels, d_model),nn.ReLU(inplace=True))
    
    def finetune(self, x):
        # Required method. Outputs extracted ECG features with shape [batch, d_model] or [batch, seq_len, d_model]
        if len(x.shape) == 2:
            x = x.unsqueeze(-1)
        x = self.encoder(x)

        return x

    def forward(self, x):
        x = self.finetune(x)
        return x

### Step 2. Create a Configuration File
Add a model configuration file in ```config/model```. Here's what each parameter means:

- ```_name_```: Model name
- ```_target_```: Location where the model is defined
- ```oa_mlp_d_model```: Output feature dimension for "one-to-all" tasks (e.g., Classification and Forecast tasks can flatten features since sequence length doesn't matter)
- ```oo_mlp_d_model```: Output feature dimension for "one-to-one" tasks (e.g., Detection and Generation tasks require keeping sequence length, so this differs from ```oa_mlp_d_model```). See the head and PSSM implementations for detailed examples.
- ```is_frozen```: Whether the Customized Model participates in fine-tuning (0 = yes, 1 = no)
- ```model_root_path```: Path where the model is saved
- ```model_save_name```: Model file name (used when ```is_finetune=False``` to load the model)

```yaml
defaults:
  - _self_

_name_ : CustomizedModel
_target_: models.CustomizedModel.CustomizedModel
input_channels: 1
d_model: 128
oa_mlp_d_model : 128
oo_mlp_d_model : 128

patch_len : 1
is_frozen : 0

model_root_path : checkpoints/CustomizedModel
model_save_name : 

is_load_pretrain_model : 0

```

### Step 3.Run validation

```shell
python val_run.py model=CustomizedModel data=AF
```